# MODEL 2: Chi-Square + Logistic Regression

**Hazırlayan:** İsmail Can Günay

## Model Özeti
- **Feature Selection:** Chi-Square Test (100 özellik)
- **Model:** Logistic Regression (L2 regularization)
- **Hedef:** Part 1-2 (Binary Classification)

## Veri Kullanımı
- **Toplam Train Verisi:** 848,024 örnek
- **Sampling (Feature Selection için):** 150,000 örnek (stratified)
- **Model Eğitimi:** Tüm train verisi (848,024 örnek) - Seçilen 100 özellik ile
- **Test Verisi:** 147,207 örnek
- **Toplam Özellik:** 1,786 (encoding sonrası)
- **Seçilen Özellik:** 100


In [1]:
# Gerekli kütüphaneleri yükle
import pandas as pd
import numpy as np
from scipy.sparse import load_npz, csr_matrix
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import joblib
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print("MODEL 2: Chi-Square + Logistic Regression")
print("Hazirlayan: Ismail Can Gunay")
print("="*70)

MODEL 2: Chi-Square + Logistic Regression
Hazirlayan: Ismail Can Gunay


In [2]:
# Temizlenmiş dataset'i yükle (tek dosya)
df_clean = pd.read_csv("C:\AzraTanriver\CrimeData\ismailcan_gunay_modeller\clean_data.csv")

# Sparse matrisi yükle (tüm özellikler)
X_clean_full = load_npz("X_clean_ready.npz")

# Target değişkeni
y_clean = df_clean['Part 1-2'].copy()
y_clean_binary = (y_clean > 0).astype(int)  # 0: Part 1, 1: Part 2

print(f"Clean data boyutu: {X_clean_full.shape}")
print(f"Target dağılımı: {pd.Series(y_clean_binary).value_counts().to_dict()}")

# Clean data'dan train ve test setlerini ayır (stratified split)
print("\nClean data'dan train ve test setleri ayrılıyor (stratified split)...")
train_indices, test_indices = train_test_split(
    df_clean.index,
    test_size=0.2,  # %20 test, %80 train
    stratify=y_clean_binary,
    random_state=123  # Model 2 için özel random seed (Model 1'den farklı)
)

# Sparse matrisi ve target'ı bu indekslere göre böl
X_train_full = X_clean_full[train_indices]
X_test_full = X_clean_full[test_indices]
y_train_binary = y_clean_binary.iloc[train_indices].values
y_test_binary = y_clean_binary.iloc[test_indices].values

# DataFrame'i de aynı indekslere göre böl
df_train = df_clean.loc[train_indices].reset_index(drop=True)
df_test = df_clean.loc[test_indices].reset_index(drop=True)

print(f"Train set boyutu: {X_train_full.shape}")
print(f"Test set boyutu: {X_test_full.shape}")
print(f"Train target dağılımı: {pd.Series(y_train_binary).value_counts().to_dict()}")

# Stratified sampling (farklı bir örnekleme stratejisi)
print("\nStratified sampling yapılıyor (150,000 örnek)...")
X_sample, _, y_sample, _ = train_test_split(
    X_train_full, 
    y_train_binary,
    train_size=150000,
    stratify=y_train_binary,
    random_state=123  # Farklı random seed (Model 1'den farklı)
)
print(f"Sampling sonrası: {X_sample.shape}")

print(f"\nVERİ KULLANIMI ÖZETİ:")
print(f"  - Toplam train verisi: {X_train_full.shape[0]:,} örnek")
print(f"  - Sampling (feature selection için): {X_sample.shape[0]:,} örnek")
print(f"  - Model eğitimi: {X_train_full.shape[0]:,} örnek (tüm train verisi)")
print(f"  - Test verisi: {X_test_full.shape[0]:,} örnek")

Clean data boyutu: (995231, 1786)
Target dağılımı: {0: 600045, 1: 395186}

Clean data'dan train ve test setleri ayrılıyor (stratified split)...
Train set boyutu: (796184, 1786)
Test set boyutu: (199047, 1786)
Train target dağılımı: {0: 480036, 1: 316148}

Stratified sampling yapılıyor (150,000 örnek)...
Sampling sonrası: (150000, 1786)

VERİ KULLANIMI ÖZETİ:
  - Toplam train verisi: 796,184 örnek
  - Sampling (feature selection için): 150,000 örnek
  - Model eğitimi: 796,184 örnek (tüm train verisi)
  - Test verisi: 199,047 örnek


In [3]:
# Chi-Square için non-negative değerler gerekli (MinMax scaling)
print("\nChi-Square için veriler normalize ediliyor (0-1 arası)...")
scaler = MinMaxScaler()
X_sample_dense = X_sample.toarray()
X_sample_scaled = scaler.fit_transform(X_sample_dense)
X_sample_scaled = csr_matrix(X_sample_scaled)  # Tekrar sparse'a çevir

# Chi-Square: Kategorik özelliklerin target ile istatistiksel bağımsızlığını test eder
print("\nChi-Square testi ile feature selection yapılıyor...")
print("Chi-Square skorları hesaplanıyor (bu biraz zaman alabilir)...")

# En iyi 100 özelliği seç
k_best = 100
chi2_selector = SelectKBest(score_func=chi2, k=k_best)
X_selected = chi2_selector.fit_transform(X_sample_scaled, y_sample)

# Seçilen özelliklerin indekslerini al
selected_indices = chi2_selector.get_support(indices=True)

# Tüm train ve test setlerine uygula (önce normalize et)
print("\nTüm veri setine normalize ve feature selection uygulanıyor...")
X_train_dense = X_train_full.toarray()
X_test_dense = X_test_full.toarray()

X_train_scaled = scaler.transform(X_train_dense)
X_test_scaled = scaler.transform(X_test_dense)

X_train_selected = csr_matrix(X_train_scaled[:, selected_indices])
X_test_selected = csr_matrix(X_test_scaled[:, selected_indices])

print(f"Seçilen özellik sayısı: {len(selected_indices)}")
print(f"Seçilen özelliklerin ilk 10 indeksi: {selected_indices[:10]}")

# Chi-Square skorlarını göster
chi2_scores = chi2_selector.scores_
valid_scores = chi2_scores[~np.isnan(chi2_scores)]
if len(valid_scores) > 0:
    top_10_indices = np.argsort(valid_scores)[-10:][::-1]
    print(f"\nEn yüksek Chi-Square skorlarına sahip 10 özellik:")
    for idx in top_10_indices[:10]:
        if not np.isnan(chi2_scores[idx]):
            print(f"  Özellik {idx}: Chi2 = {chi2_scores[idx]:.2f}")

print(f"\nVERİ KULLANIMI:")
print(f"  - Normalizasyon: {X_sample.shape[0]:,} örnek kullanıldı")
print(f"  - Feature selection: {X_sample.shape[0]:,} örnek kullanıldı")
print(f"  - Seçilen özellik sayısı: {len(selected_indices)}")
print(f"  - Train set (seçilen özelliklerle): {X_train_selected.shape}")
print(f"  - Test set (seçilen özelliklerle): {X_test_selected.shape}")


Chi-Square için veriler normalize ediliyor (0-1 arası)...

Chi-Square testi ile feature selection yapılıyor...
Chi-Square skorları hesaplanıyor (bu biraz zaman alabilir)...

Tüm veri setine normalize ve feature selection uygulanıyor...
Seçilen özellik sayısı: 100
Seçilen özelliklerin ilk 10 indeksi: [  1   3   7 126 130 165 172 571 575 672]

En yüksek Chi-Square skorlarına sahip 10 özellik:
  Özellik 3: Chi2 = 90438.00
  Özellik 1759: Chi2 = 2.49
  Özellik 1741: Chi2 = 2.23
  Özellik 1750: Chi2 = 2.13
  Özellik 1745: Chi2 = 1.67
  Özellik 1729: Chi2 = 155.49
  Özellik 1777: Chi2 = 4.08
  Özellik 1746: Chi2 = 1058.49
  Özellik 1755: Chi2 = 0.08

VERİ KULLANIMI:
  - Normalizasyon: 150,000 örnek kullanıldı
  - Feature selection: 150,000 örnek kullanıldı
  - Seçilen özellik sayısı: 100
  - Train set (seçilen özelliklerle): (796184, 100)
  - Test set (seçilen özelliklerle): (199047, 100)


In [4]:
# Logistic Regression: Doğrusal bir sınıflandırma modeli
lr_model = LogisticRegression(
    C=1.0,                    # Regularization parametresi
    penalty='l2',             # L2 regularization (Ridge)
    solver='lbfgs',           # Optimizasyon algoritması
    max_iter=1000,           # Maksimum iterasyon
    random_state=42,
    class_weight='balanced',  # Dengesiz sınıflar için ağırlıklandırma
    n_jobs=-1
)

print("Model eğitimi başlıyor...")
print(f"Eğitim verisi: {X_train_selected.shape[0]:,} örnek, {X_train_selected.shape[1]} özellik")
lr_model.fit(X_train_selected, y_train_binary)
print("Model eğitimi tamamlandı!")

# Katsayıları göster
coefficients = lr_model.coef_[0]
top_10_features = np.argsort(np.abs(coefficients))[-10:][::-1]
print(f"\nEn yüksek katsayıya sahip 10 özellik:")
for i, idx in enumerate(top_10_features):
    feat_idx = selected_indices[idx]
    print(f"  {i+1}. Özellik {feat_idx}: Coef = {coefficients[idx]:.4f}")

print(f"\nVERİ KULLANIMI:")
print(f"  - Model eğitimi: {X_train_selected.shape[0]:,} örnek kullanıldı")
print(f"  - Özellik sayısı: {X_train_selected.shape[1]}")

Model eğitimi başlıyor...
Eğitim verisi: 796,184 örnek, 100 özellik
Model eğitimi tamamlandı!

En yüksek katsayıya sahip 10 özellik:
  1. Özellik 3: Coef = 17.2317
  2. Özellik 1: Coef = -1.0457
  3. Özellik 1742: Coef = -0.9133
  4. Özellik 1760: Coef = -0.9133
  5. Özellik 1762: Coef = -0.5805
  6. Özellik 1756: Coef = -0.5087
  7. Özellik 1751: Coef = -0.3991
  8. Özellik 1746: Coef = -0.3847
  9. Özellik 1765: Coef = -0.1870
  10. Özellik 1718: Coef = -0.1870

VERİ KULLANIMI:
  - Model eğitimi: 796,184 örnek kullanıldı
  - Özellik sayısı: 100


In [5]:
# Test setine tahmin
y_pred_proba = lr_model.predict_proba(X_test_selected)[:, 1]  # Part 2 olasılığı
y_pred = lr_model.predict(X_test_selected)

print(f"Tahmin edilen sınıf dağılımı:")
print(f"  Part 1 (0): {(y_pred == 0).sum()}")
print(f"  Part 2 (1): {(y_pred == 1).sum()}")

print(f"\nVERİ KULLANIMI:")
print(f"  - Test tahminleri: {X_test_selected.shape[0]:,} örnek")

Tahmin edilen sınıf dağılımı:
  Part 1 (0): 120009
  Part 2 (1): 79038

VERİ KULLANIMI:
  - Test tahminleri: 199,047 örnek


## AŞAMA 5: SUBMISSION DOSYASI OLUŞTURMA


In [6]:
# Test setindeki DR_NO'ları al
submission_df = pd.DataFrame({
    'DR_NO': df_test['DR_NO'].values if 'DR_NO' in df_test.columns else df_test.index.values,
    'Part 1-2': y_pred
})

# Orijinal değerlere geri çevir
submission_df['Part 1-2'] = submission_df['Part 1-2'].map({
    0: -0.834220,
    1: 1.198725
})

# Dosyayı kaydet
submission_file = "ismailcan_model2_submission.csv"
submission_df.to_csv(submission_file, index=False)
print(f"[OK] Submission dosyasi kaydedildi: {submission_file}")
print(f"   Toplam tahmin sayısı: {len(submission_df)}")

# Model ve selector'ı kaydet
joblib.dump(lr_model, "ismailcan_model2_lr.pkl")
joblib.dump(chi2_selector, "ismailcan_model2_selector.pkl")
joblib.dump(selected_indices, "ismailcan_model2_selected_indices.pkl")
joblib.dump(scaler, "ismailcan_model2_scaler.pkl")

print("\n" + "="*70)
print("MODEL 2 TAMAMLANDI!")
print("="*70)
print("\nÖZET:")
print(f"  - Feature Selection: Chi-Square Test (Top {k_best} features)")
print(f"  - Model: Logistic Regression (L2 regularization)")
print(f"  - Submission: {submission_file}")
print("\nVERİ KULLANIMI ÖZETİ:")
print(f"  - Toplam train verisi: {X_train_full.shape[0]:,} örnek")
print(f"  - Sampling (feature selection): {X_sample.shape[0]:,} örnek")
print(f"  - Model eğitimi: {X_train_selected.shape[0]:,} örnek")
print(f"  - Test tahminleri: {X_test_selected.shape[0]:,} örnek")
print("="*70)

[OK] Submission dosyasi kaydedildi: ismailcan_model2_submission.csv
   Toplam tahmin sayısı: 199047

MODEL 2 TAMAMLANDI!

ÖZET:
  - Feature Selection: Chi-Square Test (Top 100 features)
  - Model: Logistic Regression (L2 regularization)
  - Submission: ismailcan_model2_submission.csv

VERİ KULLANIMI ÖZETİ:
  - Toplam train verisi: 796,184 örnek
  - Sampling (feature selection): 150,000 örnek
  - Model eğitimi: 796,184 örnek
  - Test tahminleri: 199,047 örnek
